In [4]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

def show_data_relation(dataname):
    df = pd.read_csv(dataname, index_col='Datetime')
    df = df.astype('double')
    
    count = [0, 0, 0]
    df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = df['class'].shift(-1)
    print(' LOW/MID/HIGH')
    print(count)
    sns.pairplot(df, hue='class', palette='RdYlBu', height=8, kind='reg').savefig('./data_plot'+dataname+'.png')
    
#     iris = sns.load_dataset("iris")
#     sns.pairplot(iris, hue='species', palette='RdYlBu', height=8, kind='reg', diag_kind='kde')

    
def classify(x, c, R=0.01):
        if x<=-R:
            c[0]+=1
            return 0
        elif -R<x<R:
            c[1]+=1
            return 1
        elif R<=x:
            c[2]+=1
            return 2
        
def classify_two(x, c):
        if x<=0:
            c[0]+=1
            return 0
        elif 0<x:
            c[2]+=1
            return 2

def predict(dataname):

    df = pd.read_csv(dataname, index_col='Datetime')
    count = [0,0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print(" LOW/MID/HIGH")
    print(count)
    
    
    X = df.drop('class', axis=1)
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
    
#     names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
#              "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
#              "Quadratic Discriminant Analysis"]
#     classifiers = [
#         KNeighborsClassifier(3),
#         SVC(kernel="linear", C=0.025),
#         SVC(gamma=2, C=1),
#         DecisionTreeClassifier(max_depth=5),
#         RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#         AdaBoostClassifier(),
#         GaussianNB(),
#         LinearDiscriminantAnalysis(),
#         QuadraticDiscriminantAnalysis()]
    
#     accuracy_scores = []
#     precision_scores = []
    
    
    clf = LinearDiscriminantAnalysis()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    result = clf.predict(X_test)

    print('accuracy_score: ', end='')
    print(accuracy_score(y_test, result))
    #accuracy_scores.extend([accuracy_score(y_test, result)])
    print('confusion_matrix: ')
    print(confusion_matrix(y_test, result))
    print('precision_score: ', end='')
    print(precision_score(y_test, result, average=None))
    #precision_scores.extend([precision_score(y_test, result, average=None)])

    possestion = 50000
    pay = 5000
    po_rate = 1.95

    for i in range(len(result)):
        possestion -= pay
        if (result[i]==0):#Low予想のとき
            if (result[i]==y_test[i]): #予測成功
                possestion += pay*po_rate

        elif(result[i]==2):#High予想の時
            if (result[i]==y_test[i]): #予測成功
                possestion += pay*po_rate
        else:  #Mid予想の時
            possestion += pay
    print(str(len(result))+"days trade")
    print("possestion: "+ str(possestion))

    print('')
    
    return possestion, accuracy_score(y_test, result), clf
    
#     print(accuracy_scores)
#     print(precision_scores)

def main(filename):
    #namelist = ['20101to20208_USD_JPY_D.csv','20101to20208_EUR_JPY_D.csv','20101to20208_AUD_JPY_D.csv','20101to20208_GBP_JPY_D.csv','20101to20208_NZD_JPY_D.csv','20101to20208_CAD_JPY_D.csv','20101to20208_CHF_JPY_D.csv','20101to20208_ZAR_JPY_D.csv']
    
#     for i in range(len(namelist)):
#         print(namelist[i])
#         predict(namelist[i]) 
#         #show_data_relation(namelist[i])
    best_pos = 0
    best_clf = 0
    best_acc = 0
    for i in range(1):
        possestion, accuracy, clf = predict(filename)
        if possestion>best_pos:
            best_pos = possestion
            best_clf = clf
            best_acc = accuracy
    print("possestion: "+str(best_pos))
    print("accuracy: "+str(best_acc))
    import pickle
    filename = 'LDA_01.sav'
    pickle.dump(clf, open(filename, 'wb'))
            
    

main('20101to20208_GBP_JPY_D.csv')

 LOW/MID/HIGH
[1106, 0, 1075]
accuracy_score: 0.5252293577981652
confusion_matrix: 
[[183  37]
 [170  46]]
precision_score: [0.5184136  0.55421687]
436days trade
possestion: 102750.0

possestion: 102750.0
accuracy: 0.5252293577981652
